In [1]:
import numpy as np 
import re
from scipy.interpolate import CubicSpline
from astropy.table import Table
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.constants import c
from astropy.coordinates import SkyCoord

In [2]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [3]:
#Read all the files from the SED modeling. 
folder = "../../SED_Modeling"

#Band information
bands = Table.read("{}/bandmag.dat".format(folder), format='ascii')
for i, hn in enumerate(['bname', 'bcal', 'jyzero']):
    bands.rename_column("col{}".format(i+1), hn)
bands['bname'][-2] = "g_misc"
bands['bname'][-1] = "r_misc"

#SED model parameters
d20 = Table.read("{}/double.20".format(folder), format='ascii')
for i, hn in enumerate(['i', 'redshift', 'chi2', 'vec1', 'vec2', 'vec3', 'vec4', 'vec5']):
    d20.rename_column("col{}".format(i+1), hn)
d22 = Table.read("{}/double.22".format(folder), format='ascii')
for i, hn in enumerate(['i', 'ebv1', 'ebv2', 'igm', 'wid']):
    d22.rename_column("col{}".format(i+1), hn)

#The Pran probs
pran = dict()
cat = open("{}/f_stat/comp.sort".format(folder))
for line in cat:
    x = line.split()
    pran[x[0]] = float(x[-1])

In [4]:
#Read the photometry
d21_raw = np.loadtxt("{}/double.21".format(folder))
nchan = len(bands)
d21 = Table()
d21['wid'] = d22['wid']
for k, bname in enumerate(bands['bname']):
    d21[bname] = d21_raw[k::nchan, 1] * u.Jy
    d21[bname+" mod"] = d21_raw[k::nchan, 2] * u.Jy
    d21[bname+" err"] = d21_raw[k::nchan, 3] * u.Jy

In [5]:
dbase = Table.read("{}/dbase.fits".format(folder))
coords = dict()
for wid in pran.keys():
    k = np.where(dbase['Short Name']==wid)[0][0]
    coords[wid] = SkyCoord(ra=dbase['ra'][k]*u.deg, dec=dbase['dec'][k]*u.deg)

In [6]:
coords['W0116-0505'].to_string(style='hmsdms', sep=":")

'01:16:01.411584 -05:05:04.09308'

In [7]:
#We now want to get the 6um luminosities of the unreddened best-fit quasar templates. 
seds = np.loadtxt("{}/agn_spec.dat".format(folder), skiprows=1)
agn_sed = CubicSpline(seds[:,0], seds[:,2])

In [8]:
#Calculate the 6um luminosities for each target. 
z = d20['redshift']
DL = cosmo.luminosity_distance(z)
for j, vec in enumerate(['vec1', 'vec5']):
    f_nu = d20[vec]*agn_sed(6.0) * u.Jy
    L_nu = (4.*np.pi*DL**2)/(1+z) * f_nu
    L6um = (c/(6*u.micron) * L_nu).to(u.erg/u.s)
    d22['log L6um AGN{}'.format(j+1)] = np.log10(L6um.value)

In [9]:
#Final table.
wids = ['W0019$-$1046', 'W0116$-$0505', 'W0204$-$0506', 'W0220+0137', 'W0831+0140']
k = list()
for wid in wids:
    k.append(np.argwhere(d22['wid']==re.sub("\$","",wid))[0][0])

Ftab = Table()
Ftab['Short WISE ID'] = wids

ra  = [None]*len(wids)
dec = [None]*len(wids)
for i, wid in enumerate(wids):
    hmsdms1 = coords[re.sub("\$","",wid)].to_string(style='hmsdms', sep=':', precision=1)
    hmsdms2 = coords[re.sub("\$","",wid)].to_string(style='hmsdms', sep=':', precision=2)
    ra[i]  = hmsdms2.split()[0]
    dec[i] = re.sub("-","$-$",hmsdms1.split()[1])
Ftab['R.A.'] = ra
Ftab['Dec.'] = dec

Ftab['Redshift'] = d20['redshift'][k]
Ftab['Redshift'].info.format = '7.3f'

#Ftab['rmag'] = -2.5*np.log10(d21['sdssr'][k]/(3631.*u.Jy))
#Ftab['rmag'].info.format = '5.2f'

Ftab['log L6um AGN1'] = d22['log L6um AGN1'][k]
Ftab['log L6um AGN1'].info.format = '5.2f'

Ftab['E(B-V) AGN1'] = d22['ebv1'][k] 
Ftab['E(B-V) AGN1'].info.format = '5.2f'

Ftab['log L6um AGN2'] = d22['log L6um AGN2'][k]
Ftab['log L6um AGN2'].info.format = '5.2f'

Ftab['E(B-V) AGN2'] = d22['ebv2'][k] 
Ftab['E(B-V) AGN2'].info.format = '5.2f'

Ftab['p_ran'] = np.zeros(len(wids))
for i, wid in enumerate(wids):
    Ftab['p_ran'][i] = pran[re.sub("\$","",wid)]*1e2
Ftab['p_ran'].info.format = '5.3f'


In [10]:
Ftab.write("targets.tex", format='latex', overwrite=True)